<h1>Step 1</h1>
<h2>Scraping Wiki to get DataFrame with Toronto Neighborhood ZipCodes</h2>

Importing libs:

In [2]:
import urllib.request
import  bs4 as bs
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


<h1>Scrapind with BeautifulSoup</h1>

Scraping WIKI for the list of postal codes of Canada using BeautifulSoup library:

In [3]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
table_rows = []
table = soup.find('table')
table_rows = table.find_all('tr')

Converting scraped results to dataframe:

In [4]:
l = []
for tr in table_rows[1:]:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

df = pd.DataFrame(l, columns=['PostalCode', 'Borough', 'Neighborhood'])

Replacing new line characters:

In [5]:
df.replace(to_replace=r'(\r\n|\r|\n)', value=' ', regex=True, inplace=True)

If borough is not assigned - replace with NaN:

In [6]:
df['Borough'].replace(to_replace=r'Not assigned', value=np.nan, regex=True, inplace=True)

Stripping white spaces from start and end of the line:

In [7]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

Dropping NaNs:

In [8]:
df.dropna(inplace=True)

Resetting index:

In [9]:
df = df.groupby('PostalCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()

If for some row Neighborhood is not assigned, but Borough is, assign Borough value to Neighborhood:

In [10]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 0, df['Borough'], df['Neighborhood'])

Checking shape:

In [11]:
df.shape

(103, 3)

In [12]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Saving to file

In [13]:
df.to_csv('data/df_scraped.csv', index=0)